# Eval: Sentence Transformer with question-result set

This notebook outlines the steps undertook to evaluate various other models out of the box with the test set under a QA set-up.

With 512 chunk size.

# Import libraries

In [2]:
#%pip install nltk -q
#%pip install sentence_transformers -q

import numpy as np
import pandas as pd

import nltk
from nltk.tokenize import sent_tokenize
nltk.download('punkt_tab')

from sentence_transformers import SentenceTransformer, util

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


# Global variable

In [3]:
EVALS = {}

# Helper functions

In [4]:
def split_text_into_chunks(text, max_words=512, overlap_sentences=3):
    sentences = sent_tokenize(text)  # Tokenize into sentences
    chunks = []
    start_idx = 0  # Start index of the current chunk

    while start_idx < len(sentences):
        current_chunk = []
        current_word_count = 0
        idx = start_idx

        # Build a chunk until max_words is reached
        while idx < len(sentences) and current_word_count + len(sentences[idx].split()) <= max_words:
            current_chunk.append(sentences[idx])
            current_word_count += len(sentences[idx].split())
            idx += 1  # Move to next sentence

        if current_chunk:
            chunks.append(" ".join(current_chunk))

        # Stop if the last chunk reaches the end of the text
        if idx >= len(sentences):
            break

        # Move start index forward but keep overlap
        start_idx = max(idx - overlap_sentences, start_idx + 1)

    return chunks


def create_embeddings(df, model, text_column, embedding_column):
    df[embedding_column] = None

    for idx, text in enumerate(df[text_column]):
        chunks = split_text_into_chunks(text)  # Chunk the text
        chunk_embeddings = np.array(model.encode(chunks))  # Encode all chunks (shape: [num_chunks, embedding_dim])
        df.at[idx, embedding_column] = chunk_embeddings

    return df


def evaluate_retrieval(query_emb, doc_embs, relevant_idx, top_k=5):
    # Flatten the document chunks and keep track of the document they belong to
    all_chunk_embeddings = []
    doc_chunk_mapping = []  # Mapping from chunk index to document index
    for doc_idx, doc_chunks in enumerate(doc_embs):
        for chunk_emb in doc_chunks:
            all_chunk_embeddings.append(chunk_emb)
            doc_chunk_mapping.append(doc_idx)  # Store the document index for each chunk

    # Convert to numpy array for efficient computation
    all_chunk_embeddings = np.array(all_chunk_embeddings)  # Shape: [total_chunks, 768]

    # Compute cosine similarity between query and all document chunks
    similarities = np.dot(all_chunk_embeddings, query_emb.T)  # Shape: [total_chunks,]

    # Get indices of top-k most similar chunks
    top_k_chunk_indices = np.argsort(similarities.flatten())[::-1][:top_k]

    # Find the documents corresponding to the top-k chunks
    retrieved_docs = [doc_chunk_mapping[idx] for idx in top_k_chunk_indices]

    # Count the number of times the relevant document appears in the top-k retrieved documents
    relevant_document_hits = sum(1 for doc_idx in retrieved_docs if doc_idx == relevant_idx)

    # Compute rank (based on first appearance of the relevant document)
    rank = None
    for idx, doc_idx in enumerate(retrieved_docs):
        if doc_idx == relevant_idx:
            rank = idx + 1
            break

    # Compute metrics
    is_hit = relevant_document_hits > 0  # Hit if any of the top-k chunks belong to the relevant document
    mrr = 0 if rank is None else 1 / rank

    return {
        "is_hit": is_hit,
        "mrr": mrr,
        "retrieved": retrieved_docs,
        "expected": relevant_idx
    }


def evaluate_model(df, model, query_column, query_emb_column, opinion_column, opinion_emb_column, top_k=5):
    # Step 1: Create embeddings for queries and opinions
    df = create_embeddings(df, model, query_column, query_emb_column)
    df = create_embeddings(df, model, opinion_column, opinion_emb_column)

    # Step 2: Evaluate retrieval for each query
    results_dict = {}
    for idx, query_row in df.iterrows():
        opinion_id = query_row["opinion_id"]
        query_emb = query_row[query_emb_column]
        doc_embs = df[opinion_emb_column]
        results_dict[opinion_id] = evaluate_retrieval(query_emb, doc_embs, idx, top_k=top_k)

    # Step 3: Convert results to DataFrame
    results_df = pd.DataFrame.from_dict(results_dict, orient='index').reset_index().rename(columns={'index': 'opinion_id'})

    # Step 4: Merge results with the original DataFrame
    final_df = df.merge(results_df, how="left", on="opinion_id")

    # Step 5: Calculate evaluation metrics (Hit Rate and MRR)
    model_name = model.name
    EVALS[model_name] = {}
    EVALS[model_name]["hit_rate"] = final_df["is_hit"].mean()
    EVALS[model_name]["mrr"] = final_df["mrr"].mean()

    return final_df

# Load the data

In [5]:
df = pd.read_csv("outputs/3.test.csv")
df = df[["opinion_id", "opinion_word_count", "opinion", "relevant_query_qstn"]]
df.head()

,opinion_id,opinion_word_count,opinion,relevant_query_qstn
0,8122327,241,"Benedict, D. J. Upon the testimony there- is ...",What are the contract terms determined by the ...
1,2429336,1021,154 F. Supp. 2d 906 (2000) Margaret L. SUTPHIN...,What are the legal requirements for a claim of...
2,2939558,320,IN THE COURT OF CRIMINAL APPEALS OF TEXAS NO. ...,What factors are considered in determining ine...
3,4666766,84,DISTRICT COURT OF APPEAL OF THE STATE OF FLOR...,What were the grounds for Roger Fincher's appe...
4,1359066,4985,703 P.2d 699 (1985) 108 Idaho 935 Marcilena SH...,What criteria did the Idaho Supreme Court use ...


# Evaluate model: Alibaba-NLP/gte-large-en-v1.5

In [6]:
%%time

model_name = "Alibaba-NLP/gte-large-en-v1.5"
model = SentenceTransformer(model_name, trust_remote_code=True)
model.name = model_name
result1 = evaluate_model(df, model, "relevant_query_qstn", "query_embeddings", "opinion", "opinion_embeddings")

result1.head()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/71.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.35k [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

CPU times: user 6min 32s, sys: 5.95 s, total: 6min 38s
Wall time: 7min 36s


,opinion_id,opinion_word_count,opinion,relevant_query_qstn,query_embeddings,opinion_embeddings,is_hit,mrr,retrieved,expected
0,8122327,241,"Benedict, D. J. Upon the testimony there- is ...",What are the contract terms determined by the ...,"[[-0.40575194, 0.38242242, -1.2533504, -0.0408...","[[-0.3680863, 0.31216347, -0.02005817, 0.27055...",True,1.0,"[0, 381, 270, 303, 210]",0
1,2429336,1021,154 F. Supp. 2d 906 (2000) Margaret L. SUTPHIN...,What are the legal requirements for a claim of...,"[[-0.0030804283, -1.2400904, 0.0840652, -0.061...","[[0.42036155, 0.18966155, -0.969184, -0.717158...",True,1.0,"[1, 1, 1, 15, 349]",1
2,2939558,320,IN THE COURT OF CRIMINAL APPEALS OF TEXAS NO. ...,What factors are considered in determining ine...,"[[-0.1717992, 0.04643109, -0.43701205, 0.38527...","[[0.31887192, 0.059133492, 0.92121917, 0.38830...",False,0.0,"[165, 425, 124, 425, 165]",2
3,4666766,84,DISTRICT COURT OF APPEAL OF THE STATE OF FLOR...,What were the grounds for Roger Fincher's appe...,"[[-0.5754458, 0.287126, -0.71448034, -0.323354...","[[-0.22347826, -0.09282645, 0.70723313, 0.3846...",True,1.0,"[3, 55, 377, 187, 124]",3
4,1359066,4985,703 P.2d 699 (1985) 108 Idaho 935 Marcilena SH...,What criteria did the Idaho Supreme Court use ...,"[[0.58821315, -0.092121914, -0.43259895, 0.487...","[[0.83855593, 0.20392627, -1.2022392, -0.06087...",True,1.0,"[4, 4, 4, 4, 4]",4


In [7]:
result1.to_csv("4b.eval/QA-512/3.result1.csv", index=False)
len(result1)

450

In [8]:
EVALS

{'Alibaba-NLP/gte-large-en-v1.5': {'hit_rate': 0.8222222222222222,
  'mrr': 0.7646296296296297}}

# Evaluate model: thenlper/gte-large

In [9]:
model_name = "thenlper/gte-large"
model = SentenceTransformer(model_name)
model.name = model_name
result2 = evaluate_model(df, model, "relevant_query_qstn", "query_embeddings", "opinion", "opinion_embeddings")

result2.head()

modules.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/67.9k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/619 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/670M [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

,opinion_id,opinion_word_count,opinion,relevant_query_qstn,query_embeddings,opinion_embeddings,is_hit,mrr,retrieved,expected
0,8122327,241,"Benedict, D. J. Upon the testimony there- is ...",What are the contract terms determined by the ...,"[[0.015277205, -0.01800621, -0.005011426, 0.02...","[[-0.012209993, -0.020404605, -0.010719932, 0....",True,1.0,"[0, 381, 381, 381, 381]",0
1,2429336,1021,154 F. Supp. 2d 906 (2000) Margaret L. SUTPHIN...,What are the legal requirements for a claim of...,"[[-0.009878712, -0.008002432, -0.009143257, 0....","[[-0.017438138, -0.023467613, -0.0195696, 0.01...",True,1.0,"[1, 1, 331, 331, 331]",1
2,2939558,320,IN THE COURT OF CRIMINAL APPEALS OF TEXAS NO. ...,What factors are considered in determining ine...,"[[-0.020589314, 0.0004992437, -0.0014202963, 0...","[[-0.026284987, -0.012898638, -0.0124339145, 0...",False,0.0,"[425, 165, 427, 301, 382]",2
3,4666766,84,DISTRICT COURT OF APPEAL OF THE STATE OF FLOR...,What were the grounds for Roger Fincher's appe...,"[[0.015471669, -0.029853044, -0.002207473, -0....","[[-0.0037303022, -0.014129647, -0.0053730686, ...",True,1.0,"[3, 154, 55, 428, 32]",3
4,1359066,4985,703 P.2d 699 (1985) 108 Idaho 935 Marcilena SH...,What criteria did the Idaho Supreme Court use ...,"[[-0.030766599, 0.028271204, -0.018226078, 0.0...","[[-0.029757459, 0.0010514171, 0.006418267, 0.0...",True,1.0,"[4, 4, 4, 4, 4]",4


In [10]:
result2.to_csv("4b.eval/QA-512/3.result2.csv", index=False)
len(result2)

450

In [11]:
EVALS

{'Alibaba-NLP/gte-large-en-v1.5': {'hit_rate': 0.8222222222222222,
  'mrr': 0.7646296296296297},
 'thenlper/gte-large': {'hit_rate': 0.8355555555555556,
  'mrr': 0.7533333333333333}}

# Evaluate model: nomic-ai/nomic-embed-text-v1

In [12]:
model_name = "nomic-ai/nomic-embed-text-v1"
model = SentenceTransformer(model_name, trust_remote_code=True)
model.name = model_name
result3 = evaluate_model(df, model, "relevant_query_qstn", "query_embeddings", "opinion", "opinion_embeddings")

result3.head()

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/128 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/70.9k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/2.03k [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/270 [00:00<?, ?B/s]

,opinion_id,opinion_word_count,opinion,relevant_query_qstn,query_embeddings,opinion_embeddings,is_hit,mrr,retrieved,expected
0,8122327,241,"Benedict, D. J. Upon the testimony there- is ...",What are the contract terms determined by the ...,"[[-0.008362964, 0.01585526, -0.026715895, -0.0...","[[0.032978956, -0.0062502497, -0.0033082487, -...",True,1.0,"[0, 381, 381, 381, 389]",0
1,2429336,1021,154 F. Supp. 2d 906 (2000) Margaret L. SUTPHIN...,What are the legal requirements for a claim of...,"[[0.043549, -0.021764398, -0.024917226, -0.027...","[[0.04864664, 0.0039858404, -0.009498379, -0.0...",True,1.0,"[1, 1, 1, 15, 299]",1
2,2939558,320,IN THE COURT OF CRIMINAL APPEALS OF TEXAS NO. ...,What factors are considered in determining ine...,"[[0.024418974, 0.027770717, -0.022088341, -0.0...","[[0.033952706, -0.002902296, -0.00263778, -0.0...",False,0.0,"[165, 425, 50, 347, 427]",2
3,4666766,84,DISTRICT COURT OF APPEAL OF THE STATE OF FLOR...,What were the grounds for Roger Fincher's appe...,"[[0.034150023, 0.04183146, -0.02972716, -0.032...","[[0.002662752, -0.01739174, -0.023292677, -0.0...",True,1.0,"[3, 128, 11, 428, 84]",3
4,1359066,4985,703 P.2d 699 (1985) 108 Idaho 935 Marcilena SH...,What criteria did the Idaho Supreme Court use ...,"[[0.013425105, 0.052297864, -0.014297089, -0.0...","[[0.024143236, -0.005396211, -0.012538964, -0....",True,1.0,"[4, 4, 4, 4, 4]",4


In [13]:
result3.to_csv("4b.eval/QA-512/3.result3.csv", index=False)
len(result3)

450

In [14]:
EVALS

{'Alibaba-NLP/gte-large-en-v1.5': {'hit_rate': 0.8222222222222222,
  'mrr': 0.7646296296296297},
 'thenlper/gte-large': {'hit_rate': 0.8355555555555556,
  'mrr': 0.7533333333333333},
 'nomic-ai/nomic-embed-text-v1': {'hit_rate': 0.8088888888888889,
  'mrr': 0.7347777777777778}}

# Evaluate model: BAAI/bge-m3

In [15]:
model_name = 'BAAI/bge-m3'
model = SentenceTransformer('BAAI/bge-m3', trust_remote_code=True)
model.name = model_name
result4 = evaluate_model(df, model, "relevant_query_qstn", "query_embeddings", "opinion", "opinion_embeddings")

result4.head()

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/15.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/687 [00:00<?, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

,opinion_id,opinion_word_count,opinion,relevant_query_qstn,query_embeddings,opinion_embeddings,is_hit,mrr,retrieved,expected
0,8122327,241,"Benedict, D. J. Upon the testimony there- is ...",What are the contract terms determined by the ...,"[[-0.059338793, 0.014934848, -0.00617138, -0.0...","[[-0.038305905, 0.032566477, -0.026009643, -0....",True,1.0,"[0, 205, 448, 381, 214]",0
1,2429336,1021,154 F. Supp. 2d 906 (2000) Margaret L. SUTPHIN...,What are the legal requirements for a claim of...,"[[-0.037957225, -0.019329162, -0.011700455, 0....","[[-0.049252104, 0.0102680875, -0.044354577, -0...",True,1.0,"[1, 1, 1, 15, 331]",1
2,2939558,320,IN THE COURT OF CRIMINAL APPEALS OF TEXAS NO. ...,What factors are considered in determining ine...,"[[-0.0322296, 0.026047228, -0.010783682, -0.02...","[[-0.048325725, 0.011996219, -0.028630974, -0....",False,0.0,"[165, 425, 165, 165, 50]",2
3,4666766,84,DISTRICT COURT OF APPEAL OF THE STATE OF FLOR...,What were the grounds for Roger Fincher's appe...,"[[-0.024493296, -0.042742852, -0.0035741676, 0...","[[-0.05142785, -0.0016678479, -0.016272588, -0...",True,1.0,"[3, 428, 315, 195, 144]",3
4,1359066,4985,703 P.2d 699 (1985) 108 Idaho 935 Marcilena SH...,What criteria did the Idaho Supreme Court use ...,"[[-0.036776025, -0.036311258, 0.006017427, 0.0...","[[-0.03737013, 0.013740901, -0.025323205, -0.0...",True,1.0,"[4, 4, 4, 4, 4]",4


In [16]:
result4.to_csv("4b.eval/QA-512/3.result4.csv", index=False)
len(result4)

450

In [17]:
EVALS

{'Alibaba-NLP/gte-large-en-v1.5': {'hit_rate': 0.8222222222222222,
  'mrr': 0.7646296296296297},
 'thenlper/gte-large': {'hit_rate': 0.8355555555555556,
  'mrr': 0.7533333333333333},
 'nomic-ai/nomic-embed-text-v1': {'hit_rate': 0.8088888888888889,
  'mrr': 0.7347777777777778},
 'BAAI/bge-m3': {'hit_rate': 0.7977777777777778, 'mrr': 0.7378148148148148}}